In [1]:
import pandas as pd
import numpy  as np
import os
import re
from collections import Counter

In [4]:
#要處理最後交易通路名稱
def 客戶分析(date = None):
    if date != None and date in os.listdir(os.getcwd()):
        file_list         = []
        folder_list       = []
        folder            = os.getcwd()
        particular_folder = folder + '\\' + date

        for file in os.listdir(particular_folder):
            #如果TXT_M_開頭，放到file_list
            if re.match(r'TXT_M_',file):
                file_list.append(particular_folder + '\\' + file)
            #如果日報開頭，放到file_list
            elif re.match(r'日報',file):
                folder_list.append(particular_folder + '\\' + file)

        for second_folder in folder_list:
            #把日報資料夾內的檔案設成second_level_file
            second_level_files = os.listdir(second_folder)
            for second_level_file in second_level_files:
                if re.match(r'TXT_D_',second_level_file):
                    file_list.append(second_folder + '\\' + second_level_file)



        total_df = pd.DataFrame({})
        for file in file_list:
            # columns的選擇，之後可以改
            try:
                #發現最後交易日期有兩種寫法，都先讀進來再rename
                try:
                    excel             = pd.read_excel(file,skiprows=4)[['案件號','戶號','戶名','客戶\n風險\n等級','最後交易\n通路\n名稱']]
                except:
                    excel             = pd.read_excel(file,skiprows=4)
                    if '通路名稱' in excel:
                        excel         = excel[['案件號','戶號','戶名','客戶\n風險\n等級','通路名稱']]
                    else:
                        excel         = excel[['案件號','戶號','戶名','客戶\n風險\n等級','最後交易\n通路名稱']]
                    excel.rename(columns = {'最後交易\n通路名稱':'最後交易\n通路\n名稱'})
                excel['source']   = re.search(r'#\d+R{0,1}',file)[0]
                total_df          = total_df.append(excel)
            # 客戶\n風險\n等級 && 最後交易\n通路\n名稱 不在column內的處理方式
            except:
                print(file)
                excel             = pd.read_excel(file,skiprows=4)[['案件號','契約代號','契約名稱']]
                if '客戶\n風險\n等級' not in excel.columns:
                    excel['客戶\n風險\n等級'] = ''
                if '最後交易\n通路\n名稱' not in excel.columns:
                    excel['最後交易\n通路\n名稱'] = ''
                excel['source']   = re.search(r'#\d+R{0,1}',file)[0]
                excel.columns     = ['案件號','戶號','戶名','客戶\n風險\n等級','最後交易\n通路\n名稱','source']
                total_df          = total_df.append(excel)
        #因為後面要iloc 所以reset_index
        total_df.reset_index(drop=True, inplace=True)


        all_client_list = []
        for 戶類, 資料 in total_df.groupby(['戶號','戶名']):
            
            ind_client_list    = []

            #account index
            ind_client_list.append(戶類[0])

            #account name
            ind_client_list.append(戶類[1])

            #change source list to string
            source_string_list = (' ').join(list(set(pd.DataFrame(資料)['source'])))

            #this is risk level
            ind_client_list.append(資料.iloc[0,3])
            ind_client_list.append(source_string_list)

            #add Counter
            ind_client_list.append(Counter(list(pd.DataFrame(資料)['source'])))

            #add Total Appperance Time
            ind_client_list.append(len(pd.DataFrame(資料)['source']))

            #add Final Trade Channel
            ind_client_list.append(pd.DataFrame(資料)['最後交易\n通路\n名稱'].unique()[0])

            #放入ind_client_list                  
            all_client_list.append(ind_client_list)
    
        csv_file = pd.DataFrame(all_client_list,columns = ['戶號','戶名','客戶\n風險\n等級','樣態報表' + date,'source_counter','總出現樣態次數','最後交易\n通路\n名稱'])
        csv_file.to_csv(r'output/'+date+'_client_analysis.csv',encoding='utf-8-sig',index=False,header=True)


        #### 將高風險客戶區分出來 #####
        csv_file_high_risk = csv_file[csv_file['客戶\n風險\n等級'] == '高風險']
        csv_file_high_risk.to_csv(r'output/'+date+'_client_analysis(高風險).csv',encoding='utf-8-sig',index=False,header=True)
        return (pd.DataFrame(all_client_list,columns = ['戶號','戶名','客戶\n風險\n等級','source_string','source_counter','總出現樣態次數','最後交易\n通路\n名稱']))
        
    elif date not in os.listdir(os.getcwd()):
        return '沒找到相關資料'

In [8]:
b = 客戶分析(date = '202203')

d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#18R_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#18_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#19R_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#19_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#20R_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#24R_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\TXT_M_#24_20220331_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\日報-單一\TXT_D_#17_20220301_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\日報-單一\TXT_D_#17_20220302_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\日報-單一\TXT_D_#17_20220303_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\日報-單一\TXT_D_#17_20220304_1.xlsx
d:\My Documents\andyhs\桌面\Andy\Fontaine_報表總整理\202203\日報-單一\TXT_D_#17_20220305_1.xlsx
d:\My Documents\an